In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from catboost import CatBoostClassifier

import dill

In [2]:
# dataset https://archive.ics.uci.edu/ml/datasets/Early+stage+diabetes+risk+prediction+dataset.
    
data = pd.read_csv("data/diabetes_data_upload.csv")

In [9]:
data

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,male,no,yes,no,yes,no,no,no,yes,no,yes,no,yes,yes,yes,1
1,58,male,no,no,no,yes,no,no,yes,no,no,no,yes,no,yes,no,1
2,41,male,yes,no,no,yes,yes,no,no,yes,no,yes,no,yes,yes,no,1
3,45,male,no,no,yes,yes,yes,yes,no,yes,no,yes,no,no,no,no,1
4,60,male,yes,yes,yes,yes,yes,no,yes,yes,yes,yes,yes,yes,yes,yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,39,female,yes,yes,yes,no,yes,no,no,yes,no,yes,yes,no,no,no,1
516,48,female,yes,yes,yes,yes,yes,no,no,yes,yes,yes,yes,no,no,no,1
517,58,female,yes,yes,yes,yes,yes,no,yes,no,no,no,yes,yes,no,yes,1
518,32,female,no,no,no,yes,no,no,yes,yes,no,yes,no,no,yes,no,0


In [3]:
# features

num_features = ['Age']

cat_features = ['Gender', 'Polyuria', 'Polydipsia', 
            'sudden weight loss', 'weakness', 'Polyphagia', 
            'Genital thrush', 'visual blurring', 'Itching', 
            'Irritability', 'delayed healing', 'partial paresis',
            'muscle stiffness', 'Alopecia', 'Obesity']

features = num_features + cat_features

target = 'class'

In [4]:
# from sklearn.base import BaseEstimator, TransformerMixin
# class LabelConverter(BaseEstimator, TransformerMixin):

#     def fit(self, X=None, y=None):
#         return self

#     def transform(self, X=None, y=None):
#         print(self.y)
#         y = y.map({'Positive':1, 'Negative':0}).astype(int)
#         return y

In [5]:
# Не получилось сделать это в Pipeline (в ячейке выше)
for col in data.columns.drop('Age'):
    map_dict = None
    
    if(col == 'Gender'):
        map_dict = {'Male':'male', 'Female':'female'}        
    elif(col == target):
        map_dict = {'Positive':1, 'Negative':0}
    else:
        map_dict = {'Yes':'yes', 'No':'no'}
        
    data[col] = data[col].map(map_dict)

data[target] = data[target].astype(int)

In [6]:
# split data

X_data = data[features]
y_data = data[target]

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

#save test
X_test.to_csv("data/X_test.csv", index=None)
y_test.to_csv("data/y_test.csv", index=None)

#save train
X_train.to_csv("data/X_train.csv", index=None)
y_train.to_csv("data/y_train.csv", index=None)

In [7]:
# fit model

classifier = Pipeline([
    ('classifier', CatBoostClassifier(
        random_state = 42, 
        silent=True,
        cat_features=cat_features))
])
    
classifier.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('classifier',
                 <catboost.core.CatBoostClassifier object at 0x000002344A3BF208>)],
         verbose=False)

In [8]:
with open("model/cat_boost_classifier.dill", "wb") as f:
    dill.dump(classifier, f)